In [5]:
import pandas as pd
from collections import OrderedDict
import pytz
import helper
import pnf

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

# Collect ETF data from API and create Pandas Panel
Grab Sector ETF data from Alpha Vantage API and then save each ETF's data into a separate date-indexed CSV file.

In [4]:
etf_panel = helper.get_sector_etf_panel()

/Users/nate/anaconda3/envs/julie-stav-ws/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


# Generate Trading Strategy
Create the trading strategy for a particular year. To do this, we will determine which sector ETF is the strongest performer on a monthly basis. This analysis will be done at the end of the month to inform trading for the following month. We will invest 100% of our portfolio into the best strongest ETF each month based on relative strength. At the end of the month, if the analysis shows that the strongest ETF is different that the ETF we are currently invested in, we will divest all money in the current ETF and invest 100% of capital into the ETF that has overtaken old ETF as the strongest.

In [15]:
# Run through each month in date range and get highest RS ETF for each month
start_year = 1999
end_year = 2000
monthly_start_dates, monthly_end_dates = helper.create_monthly_date_ranges(start_year, end_year)
monthly_best_performers = OrderedDict()

for idx in range(len(monthly_start_dates)):
    start_date = monthly_start_dates[idx]
    end_date = monthly_end_dates[idx]
    
    # Construct panel for this month
    panel = etf_panel.copy()
    for symbol in list(etf_panel.items):
        panel[symbol] = panel[symbol][start_date : end_date]
    panel = panel.dropna(axis=1)
        
    # Create RS matrix for this month
    rs_matrix = pnf.run_rs_matrix(panel)
    
    # Select the sector with the highest total and append to best performers dict. The key for
    # this dict is the start date of the next month
    best_sector = rs_matrix['total'].idxmax()
    if idx != (len(monthly_start_dates) - 1):
        monthly_best_performers[monthly_start_dates[idx + 1]] = best_sector
    
    print('Running RS matrix for {0} - {1}'.format(start_date, end_date), end='\r', flush=True)
    
print(monthly_best_performers)

OrderedDict([('1999-02-01', 'XLK'), ('1999-03-01', 'XLF'), ('1999-04-01', 'XLK'), ('1999-05-01', 'XLB'), ('1999-06-01', 'XLE'), ('1999-07-01', 'XLK'), ('1999-08-01', 'XLE'), ('1999-09-01', 'XLP'), ('1999-10-01', 'XLU'), ('1999-11-01', 'XLF'), ('1999-12-01', 'XLE'), ('2000-01-01', 'XLI'), ('2000-02-01', 'XLU'), ('2000-03-01', 'XLE'), ('2000-04-01', 'XLI'), ('2000-05-01', 'XLI'), ('2000-06-01', 'XLF'), ('2000-07-01', 'XLP'), ('2000-08-01', 'XLF'), ('2000-09-01', 'XLK'), ('2000-10-01', 'XLU'), ('2000-11-01', 'XLB'), ('2000-12-01', 'XLP')])
